In [1]:
import numpy as np
import pandas as pd
import math 
import os
os.chdir(r'C:\Users\user\python')
from finlab import Data


def sigmoid2(n,close):
    s = (close - close.shift(1))/close.shift(1).dropna() *60
    s = (s).apply(lambda x :  1 / (1 + math.exp(-x)))
    return s.shift(n)

def sigmoid(n,close):
    s = (close - close.shift(1)).dropna()
    s = (s).apply(lambda x :  1 / (1 + math.exp(-x)))
    return s.shift(n)

def bias(n,close):
    return close / close.rolling(n, min_periods=n).mean() -0.5

def acc(n,close):
    return close.shift(n) / (close.shift(2*n) + close) * 2  -0.5

def rsv(n,close):
    l = close.shift(2).rolling(n, min_periods=n).min()  # 阻擋重組的nan
    h = close.shift(2).rolling(n, min_periods=n).max()
    
    return (close - l) / (h - l)

In [2]:
data = Data()

stock_id='0050'

existday = data.get('存在日期')
close = data.get_adj('收盤價')[existday][stock_id].dropna()
open_ = data.get_adj('開盤價')[existday][stock_id].dropna()

In [3]:
features = {    
    'sigmoid2_1':sigmoid2(1,close),
    'sigmoid2_2':sigmoid2(2,close),
    'sigmoid2_3':sigmoid2(3,close),
    'sigmoid2_4':sigmoid2(4,close),
    'sigmoid2_5':sigmoid2(5,close),
    
    'sigmoid2_6':sigmoid2(6,close),
    'sigmoid2_7':sigmoid2(7,close),
    'sigmoid2_8':sigmoid2(8,close),
    'sigmoid2_9':sigmoid2(9,close),
    'sigmoid2_10':sigmoid2(10,close),
    
    'bias_10': bias(10,close),
    'bias_20': bias(20,close),
    'bias_60': bias(60,close),
    'bias_120': bias(120,close),
    'bias_240': bias(240,close),
    
    'acc_10': acc(10,close),
    'acc_20': acc(20,close),
    'acc_60': acc(60,close),
    'acc_120': acc(120,close),
    'acc_240': acc(240,close),

    'rsv_10': rsv(10,close),
    'rsv_20': rsv(20,close),
    'rsv_60': rsv(60,close),
    'rsv_120': rsv(120,close),
    'rsv_240': rsv(240,close),}

dataset=pd.DataFrame({})

for name, f in features.items():    
    dataset[name] = f 
    
dataset['rsv_10'] =((dataset['rsv_10']+2.5)/6)
dataset['rsv_20'] =((dataset['rsv_20']+1.5)/4)
dataset['rsv_60'] =((dataset['rsv_60']+1)/3)
dataset['rsv_120']=((dataset['rsv_120']+1)/3)
dataset['rsv_240']= (dataset['rsv_240']+0.5)/2

dataset = dataset.dropna()
dataset['Adj Open'] = open_.shift(-1)
dataset['Adj Close'] = close

In [5]:
dataset.to_pickle(r'C:\Users\user\python\Dueling DQN\3 Act\data\dqn3_1.pkl')